In [35]:
import run_model
import create_input_data

import pandas as pd

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
input_data = pd.read_csv('xgboost_input.csv')

In [37]:
X, train, test, model = run_model.run_model('2020-04-06')

/home/shen/projects/hack-the-crisis/src/forecast/run_model.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['predicted_change'] = pred
/home/shen/projects/hack-the-crisis/src/forecast/run_model.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['predicted_nbr_in_3_days'] = test['iva']*test['predicted_change']
/home/shen/projects/hack-the-crisis/src/forecast/run_model.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [38]:
results = run_model.results(input_data, test)
res_opt = results[['date','Region','iva','predicted']].sort_values(by=['Region','date'])
#res_opt.to_csv('forecast.csv')

/home/shen/projects/hack-the-crisis/src/forecast/run_model.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  utskrivningar['date'] = [str(datetime.strptime(day, '%Y-%m-%d').date()+timedelta(days=15)) for day in utskrivningar['date']]
/home/shen/projects/hack-the-crisis/src/forecast/run_model.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['date'] = [str(datetime.strptime(day, '%Y-%m-%d').date()+timedelta(days=3)) for day in results['date']]


In [39]:
A = input_data[['date','Region','iva']].sort_values(by=['date','Region'])
B = results[['date','Region','predicted','utskrivningar']].sort_values(by=['date','Region'])

In [41]:
not_in = A.merge(B,how='outer',on=['Region', 'date'])
#not_in.loc[not_in.Region == 'Region Stockholm']

In [42]:
from datetime import datetime
from datetime import timedelta
utskrivningar = input_data[['date', 'Region', 'iva']]
utskrivningar['date'] = [str(datetime.strptime(day, '%Y-%m-%d').date()+timedelta(days=10)) for day in utskrivningar['date']]
utskrivningar.columns = ['date', 'Region', 'utskrivningar']


/home/shen/projects/hack-the-crisis/venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [43]:
C = not_in.merge(utskrivningar, on=['date', 'Region'], how='left')
C.fillna(0, inplace=True)
C

,date,Region,iva,predicted,utskrivningar_x,utskrivningar_y
0,2020-03-06,Region Blekinge,0.0,0.0,0.0,0.0
1,2020-03-06,Region Dalarna,0.0,0.0,0.0,0.0
2,2020-03-06,Region Gotland,0.0,0.0,0.0,0.0
3,2020-03-06,Region Gävleborg,0.0,0.0,0.0,0.0
4,2020-03-06,Region Halland,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
709,2020-04-08,Region Västernorrland,0.0,3.0,2.0,3.0
710,2020-04-08,Region Västmanland,0.0,7.0,7.0,10.0
711,2020-04-08,Region Örebro län,0.0,11.0,7.0,9.0
712,2020-04-08,Region Östergötland,0.0,18.0,23.0,27.0


In [44]:
import math
#C = C.loc[C.Region == 'Region Stockholm']
C['iva_corrected'] = C.iva - C.utskrivningar_y
C[['date', 'Region', 'iva_corrected', 'predicted']]
#C.loc[C.Region == 'Region Dalarna']

,date,Region,iva_corrected,predicted
0,2020-03-06,Region Blekinge,0.0,0.0
1,2020-03-06,Region Dalarna,0.0,0.0
2,2020-03-06,Region Gotland,0.0,0.0
3,2020-03-06,Region Gävleborg,0.0,0.0
4,2020-03-06,Region Halland,0.0,0.0
...,...,...,...,...
709,2020-04-08,Region Västernorrland,-3.0,3.0
710,2020-04-08,Region Västmanland,-10.0,7.0
711,2020-04-08,Region Örebro län,-9.0,11.0
712,2020-04-08,Region Östergötland,-27.0,18.0


In [45]:
C.iva_corrected = [None if C.date.iloc[i] > '2020-04-05' else C.iva_corrected.iloc[i] for i in range(0, C.shape[0])]
C.predicted = [C.predicted.iloc[i] if math.isnan(C.iva_corrected.iloc[i]) else None for i in range(0, C.shape[0])]

C = C[['date','Region','iva_corrected','predicted']]
#C.loc[C.Region == 'Region Dalarna']
C

In [50]:
C.to_csv('forecast.csv', index=False)

In [51]:
C.loc[C.Region == 'Region Dalarna']

,date,Region,iva_corrected,predicted
0,2020-03-06,Region Blekinge,0.0,NaN
1,2020-03-06,Region Dalarna,0.0,NaN
2,2020-03-06,Region Gotland,0.0,NaN
3,2020-03-06,Region Gävleborg,0.0,NaN
4,2020-03-06,Region Halland,0.0,NaN
...,...,...,...,...
709,2020-04-08,Region Västernorrland,NaN,3.0
710,2020-04-08,Region Västmanland,NaN,7.0
711,2020-04-08,Region Örebro län,NaN,11.0
712,2020-04-08,Region Östergötland,NaN,18.0


In [ ]:
f = plt.figure(figsize=(15,50))

regioner = results.Region.unique()

for i in range(20):
    ax = f.add_subplot(10, 2, i+1)
    ax.plot(results[results['Region'] == regioner[i]]['date'], 
            results[results['Region'] == regioner[i]]['predicted'],
            '-ob',
            label='Prediktion'
           )
    ax.plot(results[results['Region'] == regioner[i]]['date'],
            results[results['Region'] == regioner[i]]['iva'],
            '-or',
            label='Utfall'
           )
    ax.set_title(regioner[i])
    ax.legend()

plt.show()

In [ ]:
importance = list(model.feature_importances_)
features = X.columns

feature_importance = pd.DataFrame({'feature': features, 'importance': importance})
feature_importance.sort_values('importance', ascending=False)